# Imports

In [ ]:
# Necessário pra rodar no Colab
# !pip uninstall accelerate transformers

Found existing installation: transformers 4.40.1
Uninstalling transformers-4.40.1:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.40.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.40.1


In [ ]:
# Esse também 
# !pip install accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 24.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch import nn
from torch.utils.data import Dataset, TensorDataset

In [ ]:
# Tokenizador e modelo do BERT
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Dataset de treino
link = 'https://raw.githubusercontent.com/pedroaugvsto/DeepLearning23.2/main/PROJETO/train.csv'
train = pd.read_csv(link)
train.head(5)

,text,tokenized_text,homophobia,obscene,insult,racism,sexism,xenophobia
0,tipo lomotif quero fazer final ano,"['tipo', 'lomotif', 'querer', 'fazer', 'final'...",0,0,0,0,0,0
1,carioca exaltando rio janeiro coisas dao ranço...,"['carioca', 'exaltar', 'rio', 'Janeiro', 'cois...",0,0,0,0,0,0
2,dejen tomarse fotos trabajen verdad sean ridíc...,"['dejen', 'tomarse', 'foto', 'trabajen', 'verd...",0,0,1,0,0,0
3,lembro nada sei queria fechar pau ontem kkkkkk...,"['lembro', 'nada', 'saber', 'querer', 'fechar'...",0,1,0,0,0,0
4,deve saber diferenciar gênero orientação sexua...,"['dever', 'saber', 'diferenciar', 'gênero', 'o...",0,0,1,0,0,0


In [ ]:
# Dataset de teste
link = 'https://raw.githubusercontent.com/pedroaugvsto/DeepLearning23.2/main/PROJETO/test.csv'
test = pd.read_csv(link)
test.head(5)

,text,tokenized_text,homophobia,obscene,insult,racism,sexism,xenophobia
0,caralho merda semana inteira pra escola vou de...,"['Caralho', 'Merda', 'semano', 'inteira', 'pra...",0,0,1,0,0,0
1,voc锚 diz desistiu incontrolavelmente continua ...,"['voc锚', 'dizer', 'desistir', 'incontrolavelme...",0,0,0,0,0,0
2,nom fica tranquilo vivo,"['nom', 'ficar', 'tranquilo', 'vivo']",0,0,0,0,0,0
3,beijo offgun tol puta merdaaahskdj,"['beijor', 'offgun', 'tol', 'puta', 'merdaaahs...",0,1,0,0,0,0
4,raiva vsf,"['raiva', 'vsf']",0,0,0,0,0,0


# Preparo dos dados

## Dataset de Treino

In [ ]:
# Tokenização e conversão pra tensores
inputs = tokenizer(train['tokenized_text'].tolist(), truncation=True, padding=True, max_length=128)
input_ids = torch.tensor(inputs['input_ids'])
attention_mask = torch.tensor(inputs['attention_mask'])

In [ ]:
# Preparo dos rótulos
labels = train[['homophobia', 'obscene', 'insult', 'racism', 'sexism', 'xenophobia']].values
labels = torch.tensor(labels, dtype=torch.float)

In [ ]:
# Criação do dataset
train_dataset = TensorDataset(input_ids, attention_mask, labels)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, tensor_dataset):
        self.tensor_dataset = tensor_dataset

    def __getitem__(self, idx):
        input_ids, attention_mask, labels = self.tensor_dataset[idx]
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

    def __len__(self):
        return len(self.tensor_dataset)

# Convertendo TensorDataset para CustomDataset
train_dataset = CustomDataset(train_dataset)

In [ ]:
print(train_dataset[0])

{'input_ids': tensor([  101,  1031,  1005,  5955,  2080,  1005,  1010,  1005,  8840, 18938,
        10128,  1005,  1010,  1005, 10861, 14544,  1005,  1010,  1005,  6904,
         6290,  1005,  1010,  1005,  2345,  1005,  1010,  1005,  2019,  2080,
         1005,  1033,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

## Dataset de Validação

In [ ]:
# Tokenização e conversão pra tensores
inputs = tokenizer(test['tokenized_text'].tolist(), truncation=True, padding=True, max_length=128)
input_ids = torch.tensor(inputs['input_ids'])
attention_mask = torch.tensor(inputs['attention_mask'])

In [ ]:
# Preparo dos rótulos
labels = test[['homophobia', 'obscene', 'insult', 'racism', 'sexism', 'xenophobia']].values
labels = torch.tensor(labels, dtype=torch.float)

In [ ]:
# Criação do dataset
test_dataset = TensorDataset(input_ids, attention_mask, labels)

In [ ]:
test_dataset = CustomDataset(test_dataset)

In [ ]:
print(test_dataset[0])

{'input_ids': tensor([  101,  1031,  1005, 14418, 28061,  1005,  1010,  1005, 21442,  2850,
         1005,  1010,  1005,  7367,  2386,  2080,  1005,  1010,  1005, 20014,
        21302,  1005,  1010,  1005, 10975,  2050,  1005,  1010,  1005,  9686,
        26289,  1005,  1010,  1005, 20868,  1005,  1010,  1005, 14866,  2595,
         2080,  1005,  1010,  1005,  9253, 21748,  2080,  1005,  1010,  1005,
         1047, 19658, 19658, 19658, 19658,  2243,  1005,  1010,  1005,  9353,
        28040, 17119,  1005,  1010,  1005,  4012, 14031,  1005,  1033,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

# Treinamento do Modelo

In [ ]:
# Argumentos de treinamento
training_args = TrainingArguments(
    output_dir='./drive/MyDrive/BERT/results/val',          # Diretório de saída para os resultados do modelo
    num_train_epochs=10,              # Número total de épocas de treinamento
    per_device_train_batch_size=16,  # Tamanho do lote de treinamento por dispositivo
    per_device_eval_batch_size=64,   # Tamanho do lote de avaliação por dispositivo
    warmup_steps=500,                # Número de etapas de aquecimento para o otimizador
    weight_decay=0.01,               # Decaimento de peso para aplicar
    logging_dir='./drive/MyDrive/BERT/logs/val',            # Diretório de saída para os logs de treinamento
)

In [ ]:
# Inicializar o Trainer
trainer = Trainer(
    model=model,                         # O modelo a ser treinado
    args=training_args,                  # Argumentos de treinamento
    train_dataset=train_dataset,         # Conjunto de dados de treinamento
    eval_dataset=test_dataset,
)

In [ ]:
# Check the type of the dataset
print("Type of test_dataset:", type(test_dataset))

# Check the length of the dataset
print("Length of test_dataset:", len(test_dataset))

# Check the type and value of the first item in the dataset
first_item = test_dataset[0]
print("Type of first item in test_dataset:", type(first_item))
print("First item in test_dataset:", first_item)

Type of test_dataset: <class '__main__.CustomDataset'>
Length of test_dataset: 6988
Type of first item in test_dataset: <class 'dict'>
First item in test_dataset: {'input_ids': tensor([  101,  1031,  1005, 14418, 28061,  1005,  1010,  1005, 21442,  2850,
         1005,  1010,  1005,  7367,  2386,  2080,  1005,  1010,  1005, 20014,
        21302,  1005,  1010,  1005, 10975,  2050,  1005,  1010,  1005,  9686,
        26289,  1005,  1010,  1005, 20868,  1005,  1010,  1005, 14866,  2595,
         2080,  1005,  1010,  1005,  9253, 21748,  2080,  1005,  1010,  1005,
         1047, 19658, 19658, 19658, 19658,  2243,  1005,  1010,  1005,  9353,
        28040, 17119,  1005,  1010,  1005,  4012, 14031,  1005,  1033,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0, 

In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
trainer.train()

model.save_pretrained('./drive/MyDrive/BERT/trained')

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
checkpoint_10000 = './drive/MyDrive/BERT/results/checkpoint-10000'
model = BertForSequenceClassification.from_pretrained(checkpoint_10000)

In [ ]:
trainer.evaluate()

{'eval_loss': 0.34648650884628296,
 'eval_runtime': 48.2665,
 'eval_samples_per_second': 144.779,
 'eval_steps_per_second': 2.279}

In [ ]:
checkpoint_11000 = './drive/MyDrive/BERT/results/checkpoint-11000'
model = BertForSequenceClassification.from_pretrained(checkpoint_11000)

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3727750778198242,
 'eval_runtime': 48.3583,
 'eval_samples_per_second': 144.505,
 'eval_steps_per_second': 2.275}

In [ ]:
checkpoint_12500 = './drive/MyDrive/BERT/results/checkpoint-12500'
model = BertForSequenceClassification.from_pretrained(checkpoint_12500)

In [ ]:
trainer.evaluate()


{'eval_loss': 0.38607388734817505,
 'eval_runtime': 48.2932,
 'eval_samples_per_second': 144.7,
 'eval_steps_per_second': 2.278}